In [98]:
import urllib.request
from lxml import html
import os

def get_text_from_article(url, xpath):
    response = urllib.request.urlopen(url)
    page_content = response.read()
    response.close()
    tree = html.fromstring(page_content)
    text = tree.xpath(xpath)
    return " ".join(text).strip()

def get_wp_articles_urls(url):
    response = urllib.request.urlopen(url)
    page_content = response.read()
    response.close()
    tree = html.fromstring(page_content)
    xpath = "//div[@class='akWTRfRe a3brEpa-']//a"
    links = tree.xpath(xpath)
    counter = 0
    urls = []
    for link in links:
        if counter % 2 == 0:
            urls.append(link.get('href'))
        counter+=1
    return urls
            
def create_wp_files(urls, xpath, folder_name):
    os.makedirs(folder_name, exist_ok=True)
    counter = 1
    for url in urls:
        text = get_text_from_article(url, xpath)
        file_name = os.path.join(folder_name,str(counter) + '.txt')
        counter+=1
        with open(file_name, 'w', encoding='utf-8') as file:
            file.write(text)

def get_text_from_wp(url, xpath, folder_name):
    try:
        urls = get_wp_articles_urls(url)
        page = 2
        while True:
            new_url = url + '/' + str(page)
            page+=1
            for url1 in get_wp_articles_urls(new_url):
                urls.append(url1)
    
    except Exception as e:
        create_wp_files(urls, xpath, folder_name)

xpath_wp = '//article//h1//text() | //article//li//text() | //article//p//text() | //article//h2//text()'
url_przestepstwa = 'https://wiadomosci.wp.pl/tag/przest%C4%99pstwa'
url_katastrofa = 'https://wiadomosci.wp.pl/tag/katastrofa'
#get_text_from_wp(url_przestepstwa, xpath_wp, 'przestępstwa')
get_text_from_wp(url_katastrofa, xpath_wp, 'katastrofa')